寻找难例子

In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm

path = "ismt_c/2400"
file_list = os.listdir(path)
result_index = "ismt_c_Result"
timeout_index = "ismt__Timeout"
for filename in tqdm(file_list, desc = 'ismt_c'):
    with open(os.path.join(path, filename)) as f:
        while f.readline():
            result = 0
            info = f.readline()
            while info[:3] != '../':
                if info[:3] == 'sat':
                    result = 1
                info = f.readline()               
            if result == 0:
                info = info.split(' : ')[0]
                info = '/'.join(info.split('/')[3:])
                # print(info)
                if info:
                    with open('hardSample.log','a') as f1:
                        print(info, file = f1)
                # else:
                #     with open('error.log','a') as f1:
                #         print(info, file = f1)

将难例子放到一个文件夹下

In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm
import shutil

len = 0
with open('hardSample.log') as f:
    filename = f.readline()
    len += 1
    while filename:
        old_file_path = '../../QF_NIA/' + filename[:-1]
        new_file_path = 'hardSample/' + filename[:-1].split('/')[-1]
        try:
            shutil.copyfile(old_file_path, new_file_path)
        finally:
            filename = f.readline()

In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm

records = pd.read_excel(r"result_base.xlsx", header=[0], index_col = [0])
# print(records)
path = "hardSample"
file_list = os.listdir(path)
for filename in tqdm(file_list, desc = 'ismt_c'):
    records = records.append({'filename': filename}, ignore_index=True, verify_integrity=True)
records.to_excel('result_base1.xlsx')

整理难例子求解数据

In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm

records = pd.read_excel(r"result_base1.xlsx", header=[0], index_col = [0])
path = "result"
file_list = os.listdir(path)
for filename in tqdm(file_list, desc = 'ismt_c'):
    result_index = filename.split('.')[0] + '_result'
    timeout_index = filename.split('.')[0] + '_timeout'
    with open(os.path.join(path, filename)) as f:
        while f.readline():
            result = 0
            info = f.readline()
            while info[:4] != 'hard':
                if info[:3] == 'sat':
                    result = 1
                elif not info:
                    result = 2
                    break
                info = f.readline()               
            if result != 2:
                name = info.split(' : ')[0].split('/')[-1]
                timeout = info.split(' : ')[1][:-5]
                # print(info)
                records.loc[records['filename'] == name, [result_index, timeout_index]] = [result, timeout]
records.to_excel('result.xlsx')

In [8]:
import pandas as pd
import os
import re
from tqdm import tqdm

records = pd.read_excel(r"result.xlsx", header=[0], index_col = [0])
for index, row in records.iterrows():
    # print(index,type(row),row['code'],row['name'])
    flag = 0
    for i in range(1, 65):
        idx = str(i) + '_result'
        # print(row[index])
        # print(type(row[index]))
        if row[idx] != 'NaN':
            if row[idx] == 1:
                flag = i
                break
    records.loc[records['filename'] == row['filename'], ['resolved']] = [flag]
records.to_excel('result1.xlsx')